In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Activation, RepeatVector, Dropout, Dense
from keras.layers import LSTM
from keras.layers.wrappers import TimeDistributed
import numpy as np
import os 

# Suppress the TensorFlow warning
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

2021-12-15 23:23:07.630961: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kacem/usr/lib64
2021-12-15 23:23:07.631001: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Function to convert integer input to vectors for RNN using one hot encoding

def encode(X,seq_len, vocab_size):
    x = np.zeros((len(X),seq_len, vocab_size), dtype=np.float32)
    for ind,batch in enumerate(X):
        for j, elem in enumerate(batch):
            x[ind, j, elem] = 1
    return x

In [3]:
# Generator function to generate infinite-stream of inputs for training

def batch_gen(batch_size=32, seq_len=10, max_no=100):
    # Randomly generate a batch of integer sequences (X) and its sorted counterpart (Y)
    x = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)
    y = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)

    while True:
	# Generates a batch of input
        X = np.random.randint(max_no, size=(batch_size, seq_len))

        Y = np.sort(X, axis=1)

        for ind,batch in enumerate(X):
            for j, elem in enumerate(batch):
                x[ind, j, elem] = 1

        for ind,batch in enumerate(Y):
            for j, elem in enumerate(batch):
                y[ind, j, elem] = 1

        yield x, y
        x.fill(0.0)
        y.fill(0.0)

In [4]:
def create_model(seq_len, max_no, n_layers, hidden_size):
    model = Sequential()
    # the encoder LSTM
    model.add(LSTM(hidden_size, input_shape=(seq_len, max_no)))
    # in next layer, repeat the input seq_len times
    model.add(RepeatVector(seq_len))
    # decoder RNN, which will return output sequence
    for _ in range(n_layers):
        model.add(LSTM(hidden_size, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(max_no)))
    model.add(Activation('softmax'))
    return model

In [5]:
# Global parameters.
batch_size=32
seq_len = 15 # number of elements in sequence to sort
max_no = 100 # upper range of the numbers in sequence
hidden_size = 128
n_layers = 2
epochs = 100000

model = create_model(seq_len, max_no, n_layers, hidden_size)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2021-12-15 23:23:10.641355: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-12-15 23:23:10.641482: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kacem/usr/lib64
2021-12-15 23:23:10.641551: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kacem/usr/lib64
2021-12-15 23:23:10.641622: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRA

In [6]:
# Now the training loop, we'll sample input batches from the generator function written previously and feed it to the RNN for learning

for ind,(X,Y) in enumerate(batch_gen(batch_size, seq_len, max_no)):
    loss, acc = model.train_on_batch(X, Y)
    # We'll test RNN after each 500 iteration to check how well it is performing
    if ind % 500 == 0:
        testX = np.random.randint(max_no, size=(1, seq_len))
        test = encode(testX, seq_len, max_no)
        #print(testX, ': input sequence')
        print(ind, ': epochs')
        print(acc, ': accuracy')
        loss_str = '{:4.3}'.format(loss)
        acc_str = '{:4.3}'.format(acc)
        
        y = model.predict(test, batch_size=1)
        np_sorted = np.sort(testX)[0]
        rnn_sorted = np.argmax(y, axis=2)[0]
        is_equal = np.array_equal(np_sorted, rnn_sorted)
        print(np_sorted, ': sorted by NumPy algorithm')
        print(rnn_sorted, ': sorted by trained RNN')
        print("\n")
        
        if acc > 0.995 :
            model.save('model.h5')
            break
        
    if ind > epochs:
        # Save trained model
        model.save('model.h5')
        break
    

0 : epochs
0.0062500000931322575 : accuracy
[ 2  3  8 15 21 23 23 46 59 60 60 68 73 84 87] : sorted by NumPy algorithm
[56 56 56 56 56 56 56 56 56 56 46 46 46 46 46] : sorted by trained RNN


500 : epochs
0.10000000149011612 : accuracy
[10 13 15 17 37 38 53 58 60 63 78 82 88 90 94] : sorted by NumPy algorithm
[ 0  8 15 24 31 40 50 58 63 67 73 78 85 96 99] : sorted by trained RNN


1000 : epochs
0.12083332985639572 : accuracy
[ 2  5  8 10 25 28 39 41 47 49 65 73 81 87 88] : sorted by NumPy algorithm
[ 0  5  9 14 20 31 36 38 44 49 66 71 81 86 90] : sorted by trained RNN


1500 : epochs
0.23749999701976776 : accuracy
[ 3 22 24 26 37 44 53 54 55 58 72 80 85 94 94] : sorted by NumPy algorithm
[ 6 20 25 28 35 45 51 53 56 58 71 78 83 93 98] : sorted by trained RNN




KeyboardInterrupt: 